In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from os import path
import os

import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from omegaconf import OmegaConf, open_dict

from experanto.datasets import ChunkDataset, SimpleChunkedDataset
from experanto.utils import LongCycler, MultiEpochsDataLoader
from experanto.dataloaders import get_multisession_dataloader

In [2]:
cfg = {'dataset':{
        'global_sampling_rate': 8,
        'global_chunk_size': 32,
        'modality_config':{
            'screen': {
                    'sampling_rate': None,
                    'chunk_size': None,
                    'valid_condition': {
                        'tier': 'train',
                        'stim_type': 'stimulus.Frame', #include both images and videos
                        'stim_type': 'stimulus.Clip'
                    },
                    'offset': 0,
                    'sample_stride': 4,
                    # necessary for the allen dataset since there are blanks after every stimuli because else no valid times are found
                    'include_blanks': True, 
                    'transforms': {
                        'ToTensor': {
                            '_target_': 'torchvision.transforms.ToTensor'
                        },
                        'Normalize': {
                            '_target_': 'torchvision.transforms.Normalize',
                            'mean': 80.0,
                            'std': 60.0
                        },
                        'Resize': {
                            '_target_': 'torchvision.transforms.Resize',
                            'size': [144, 256]
                        },
                        'CenterCrop': {
                            '_target_': 'torchvision.transforms.CenterCrop',
                            'size': 144
                        }
                    },
                    'interpolation': {}
                },
                'responses': {
                    'sampling_rate': None,
                    'chunk_size': None,
                    'offset': 0.1,
                    'transforms': {
                        'standardize': True
                    },
                    'interpolation': {
                        'interpolation_mode': 'nearest_neighbor'
                }
            }
        }
    },
    'dataloader':{
        'batch_size': 32,
        'shuffle': True,
        'num_workers': 1,
        'pin_memory': True,
        'drop_last': True,
        'prefetch_factor': 1
    }
}
cfg

{'dataset': {'global_sampling_rate': 8,
  'global_chunk_size': 32,
  'modality_config': {'screen': {'sampling_rate': None,
    'chunk_size': None,
    'valid_condition': {'tier': 'train', 'stim_type': 'stimulus.Clip'},
    'offset': 0,
    'sample_stride': 4,
    'include_blanks': True,
    'transforms': {'ToTensor': {'_target_': 'torchvision.transforms.ToTensor'},
     'Normalize': {'_target_': 'torchvision.transforms.Normalize',
      'mean': 80.0,
      'std': 60.0},
     'Resize': {'_target_': 'torchvision.transforms.Resize',
      'size': [144, 256]},
     'CenterCrop': {'_target_': 'torchvision.transforms.CenterCrop',
      'size': 144}},
    'interpolation': {}},
   'responses': {'sampling_rate': None,
    'chunk_size': None,
    'offset': 0.1,
    'transforms': {'standardize': True},
    'interpolation': {'interpolation_mode': 'nearest_neighbor'}}}},
 'dataloader': {'batch_size': 32,
  'shuffle': True,
  'num_workers': 1,
  'pin_memory': True,
  'drop_last': True,
  'prefetch_f

In [3]:
parent_folder = '../data/allen_data'
full_paths = [f.path for f in os.scandir(parent_folder) if f.is_dir()]

for subfolder in full_paths:
    print(subfolder)

../data/allen_data/experiment_951980473
../data/allen_data/experiment_951980471


In [4]:
train_dl = get_multisession_dataloader(full_paths[:2], cfg)

/usr/local/lib/python3.9/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [6]:
train_dl.loaders

{'951980473': <experanto.utils.MultiEpochsDataLoader at 0x79bd2c119550>,
 '951980471': <experanto.utils.MultiEpochsDataLoader at 0x79bbf8fd5460>}

ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 